# Neal's Funnel Target Practice

In [1]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()

  Activating project at `~/Documents/PhD/MicroCanonicalHMC.jl/examples`
Precompiling project...
  ✓ Symbolics
  ✓ Symbolics → SymbolicsForwardDiffExt
  ✓ SymbolicAnalysis
  ✓ OptimizationBase
  ✓ OptimizationBase → OptimizationFiniteDiffExt
  ✓ OptimizationBase → OptimizationForwardDiffExt
  ✓ OptimizationBase → OptimizationTrackerExt
  ✓ Optimization
  ✓ OptimizationOptimJL
  ✓ Turing
  ✓ Turing → TuringOptimExt
  11 dependencies successfully precompiled in 81 seconds. 329 already precompiled.


In [2]:
# The statistical inference frame-work we will use
using Turing
using LinearAlgebra
using Random
#using StatsPlots
using PyPlot
using Distributed

# Extra deps
using LogDensityProblemsAD, LogDensityProblems

using Revise
using MicroCanonicalHMC

[ Info: Precompiling Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0]
┌ Warning: The call to compilecache failed to create a usable precompiled cache file for SymbolicAnalysis [4297ee4d-0239-47d8-ba5d-195ecdf594fe]
│   exception = Required dependency Rmath_jll [f50d1b31-88e8-58de-be2c-1cc44531875f] failed to load from a cache file.
└ @ Base loading.jl:1992
[ Info: Skipping precompilation since __precompile__(false). Importing Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0].
[ Info: Precompiling ForwardDiff [f6369f11-7733-5829-9624-2563aa707210]
Precompiling DistributionsAD
  ✓ Distributions → DistributionsDensityInterfaceExt
  ✓ DistributionsAD
  2 dependencies successfully precompiled in 4 seconds. 82 already precompiled.
[ Info: Precompiling DistributionsAD [ced4e74d-a319-5a8a-b0ac-84af2272839c]
[ Info: Precompiling SpecialFunctionsChainRulesCoreExt [9eb7bdd4-e44c-55fc-b9cc-1a32cb715188]
Precompiling StatsFunsChainRulesCoreExt
  ✓ StatsFuns → StatsFunsInverseFunctionsExt
  ✓ StatsFuns → S

In [ ]:
d = 21
@model function funnel()
    θ ~ Truncated(Normal(0, 3), -3, 3)
    z ~ MvNormal(zeros(d-1), exp(θ)*I)
    x ~ MvNormal(z, I)
end

@model function true_funnel()
    θ ~ Truncated(Normal(0, 3), -3, 3)
    z ~ MvNormal(zeros(d-1), I)
    zz = z .* exp(θ/2)
    x ~ MvNormal(zz, I)
    
end


In [ ]:
Random.seed!(1)
(;x) = rand(funnel() | (θ=0,))
funnel_model = funnel() | (;x)

Random.seed!(1)
(;x) = rand(true_funnel() | (θ=0,))
true_model = true_funnel() | (;x)

## MCHMC

In [ ]:
mchmc = MCHMC(10_000, 0.01; adaptive=true)
espl = externalsampler(mchmc)

In [ ]:
samples = sample(funnel_model, espl, 10_000);

In [8]:
theta_mchmc = [samples.value.data[i, 1, :][1] for i in axes(samples.value.data)[1]]
x10_mchmc = [samples.value.data[i, 10+1, :][1] for i in axes(samples.value.data)[1]];

### Using the Sample interface

In [ ]:
target = TuringTarget(funnel_model)
ssamples = Sample(mchmc, target, 100_000)

In [ ]:
println(mean(ssamples[1, :]), " ", std(ssamples[1, :]))
println(mean(ssamples[11, :]), " ", std(ssamples[11, :]))

In [ ]:
println(mean(theta_mchmc), " ", std(theta_mchmc))
println(mean(x10_mchmc), " ", std(x10_mchmc))

## NUTS

In [ ]:
samples_hmc = sample(funnel_model, NUTS(10, 0.95), 50_000, progress=true; save_state=true)

In [ ]:
theta_hmc = [samples_hmc.value.data[i, 1, :][1] for i in axes(samples_hmc.value.data)[1]]
x10_hmc = [samples_hmc.value.data[i, 10+1, :][1] for i in axes(samples_hmc.value.data)[1]];


In [ ]:
truth_hmc = sample(true_model, NUTS(10, 0.95), 50_000, progress=true; save_state=true)

In [ ]:
true_theta_hmc = [truth_hmc.value.data[i, 1, :][1] for i in axes(truth_hmc.value.data)[1]]
true_x10_hmc = [truth_hmc.value.data[i, 10+1, :][1] * exp(truth_hmc.value.data[i, 1, :][1]/2) for i in axes(truth_hmc.value.data)[1]];


## Comp

In [ ]:
fig, axis = plt.subplots(2, 2, figsize=(8,8))
fig.suptitle("Neal's Funnel Comp.", fontsize=16)

fig.subplots_adjust(hspace=0)
fig.subplots_adjust(wspace=0)

axis[1,1].hist(x10_mchmc, bins=100, density=true, range=[-6,2], alpha = 0.3, label="MCHMC")
axis[1,1].hist(x10_hmc, bins=100, density=true, range=[-6,2], alpha = 0.3, label="NUTS")
axis[1,1].hist(true_x10_hmc, bins=100, density=true, range=[-6,2], alpha = 0.3, label="NUTS - Truth")
axis[1,1].legend()
axis[1,1].set_yticks([])

axis[2,2].hist(theta_mchmc, bins=100, density=true, orientation="horizontal", range=[-6, 2], alpha = 0.3)
axis[2,2].hist(theta_hmc, bins=100, density=true, orientation="horizontal", range=[-6, 2], alpha = 0.3)
axis[2,2].hist(true_theta_hmc, bins=100, density=true, orientation="horizontal", range=[-6,2], alpha = 0.3)
axis[2,2].set_xticks([])
axis[2,2].set_yticks([])

axis[1,2].hist2d(true_x10_hmc, true_theta_hmc, bins=100, range=[[-6,2],[-4, 2]])
axis[1,2].set_xlabel("x10")
axis[1,2].set_ylabel("theta")
axis[1,2].set_title("NUTS")

axis[2,1].hist2d(x10_mchmc, theta_mchmc, bins=100, range=[[-6,2],[-4, 2]])
axis[2,1].set_xlabel("x10")
axis[2,1].set_ylabel("theta")
axis[2,1].set_title("MCHMC")